# Using MNIST model on our images and drawings

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model

## Renaming loaded files

In [ ]:
def rename_files(folder_path, template):
    files = os.listdir(folder_path)

    files = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]

    files.sort()

    for i, file in enumerate(files):
        _, ext = os.path.splitext(file)
        new_name = f"{template}{i+1:04d}{ext}"
        os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_name))

In [ ]:
webcam_path = 'data/webcam'
drawing_path = 'data/drawings'

In [ ]:
rename_files(webcam_path, 'img_')
rename_files(drawing_path, 'img_')

In [ ]:
webcam_pics = os.listdir(webcam_path)
drawing_pics = os.listdir(drawing_path)

## Processing webcam images

In [ ]:
def preprocess_webcam_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    _, img_thresh = cv2.threshold(img, 135, 255, cv2.THRESH_BINARY)
    img_norm = img_thresh / 255.0

    return img_norm

In [ ]:
# Preprocess webcam images
processed_webcam_images = []
for file in webcam_pics:
    image_path = os.path.join(webcam_path, file)
    processed_image = preprocess_webcam_image(image_path)
    processed_webcam_images.append(processed_image)

In [ ]:
plt.figure(figsize=(10, 5))

num_images_to_display = 5  # Change this to the number of images you want to display
for i in range(num_images_to_display):
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(processed_webcam_images[i], cmap=plt.get_cmap('gray'))
    plt.axis('off')

plt.show()

## Processing drawings

In [ ]:
def preprocess_drawing_image(path):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    if not any(path.lower().endswith(ext) for ext in valid_extensions):
        print(f"Skipping non-image file: {path}")
        return None
    
    # Attempt to read the image
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    
    # Check if the image has an alpha channel
    if img.shape[2] == 4:
        alpha_channel = img[:, :, 3]
        
        # Replace transparent pixels with white
        img[alpha_channel == 0] = [255, 255, 255, 255]
        
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    img_resized = cv2.resize(img_gray, (28, 28))
    
    _, img_thresh = cv2.threshold(img_resized, 127, 255, cv2.THRESH_BINARY)
    
    img_normalized = img_thresh / 255.0

    img_inverted = abs(1 - img_normalized)

    return img_inverted

In [ ]:
# Preprocess drawing images
processed_drawing_images = []
for file in drawing_pics:
    image_path = os.path.join(drawing_path, file)
    processed_image = preprocess_drawing_image(image_path)
    processed_drawing_images.append(processed_image)

In [ ]:
plt.figure(figsize=(20, 5))

num_images_to_display = len(processed_drawing_images)

for i in range(num_images_to_display):
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(processed_drawing_images[i], cmap=plt.get_cmap('gray'))
    plt.axis('off')

plt.show()

In [ ]:
drawing_labels = ["6", "1", "8", "9", "3", "3", "5", "3", "6", "4", "4", "7", "3", "2", "9", "4"]

In [ ]:
plt.figure(figsize=(20, 5))

num_images_to_display = len(processed_drawing_images)

# Define labels for the drawings
drawing_labels = ["Label1", "Label2", "Label3", ...]  # Replace with your actual labels

for i in range(num_images_to_display):
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(processed_drawing_images[i], cmap=plt.get_cmap('gray'))
    plt.title(f"Prediction: {drawing_predictions[i]}\nLabel: {drawing_labels[i]}", fontsize=12)  # Include the prediction and label in the title
    plt.axis('off')

plt.show()

## Testing my MNIST model on my drawings

In [ ]:
model_path = 'mnist_simple.keras'

model = load_model(model_path)

In [ ]:
drawing_predictions = []

for drawing_image in processed_drawing_images:
    # Flatten and normalize the drawing image
    drawing_image = drawing_image.reshape(1, 28*28)

    # Make predictions on the drawing image
    predictions = model.predict(drawing_image)

    # Interpret the predictions
    predicted_class = np.argmax(predictions)
    drawing_predictions.append(predicted_class)

In [ ]:
plt.figure(figsize=(20, 5))

num_images_to_display = len(processed_drawing_images)

for i in range(num_images_to_display):
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(processed_drawing_images[i], cmap=plt.get_cmap('gray'))
    
    # Include the prediction and actual label in the title
    plt.title(f"Pred: {drawing_predictions[i]}\nReal: {drawing_labels[i]}", fontsize=12)
    
    plt.axis('off')

plt.show()